# Évaluation de l'amélioration des données

Dans ce fichier, on cherche a évaluer les compétences du NLP, dans l'objectif de calculer sa précision et son rappel. Ceux-ci sont calculé dans `statistics.ipynb`

**Librairies et données**

In [16]:
import pandas as pd
import os
import random

import matplotlib.pyplot as plt
import numpy as np
import edsnlp, edsnlp.pipes as eds

from spacy import displacy
from datetime import datetime
from IPython.display import clear_output

input_dir = "../data_clean/"
visuals_dir = "../visuals/" 

In [17]:
assert os.path.exists(input_dir), f"Il faut exéctuer le script de récolte et nettoyage (clean_data.ipynb) avant de lancer ce notebook ({input_dir} n'existe pas)."

try:
    df_person = pd.read_pickle(os.path.join(input_dir, 'df_person.pkl'))
    df_bio = pd.read_pickle(os.path.join(input_dir, 'df_bio.pkl'))
    df_note = pd.read_pickle(os.path.join(input_dir, 'df_note.pkl'))
    df_visit = pd.read_pickle(os.path.join(input_dir, 'df_visit.pkl'))
    df_condition = pd.read_pickle(os.path.join(input_dir, 'df_condition.pkl'))
    df_facteur_risque = pd.read_pickle(os.path.join(input_dir, 'df_facteur_risque.pkl'))
except FileNotFoundError as e:
    raise FileNotFoundError(f"Il faut exéctuer le script de récolte et nettoyage (clean_data.ipynb) avant de lancer ce notebook ({e.filename} n'existe pas).")

**Évaluation du NLP pour les fumeurs**

In [18]:
total = 0

faux_positif, faux_negatif = 0, 0
fumeurs, non_fumeurs = 0, 0
true_positif, true_negatif = 0, 0

echantillons = list(range(len(df_person)))
random.shuffle(echantillons)

if 'fumeur' not in df_facteur_risque.columns:
    raise ValueError("La colonne 'fumeur' n'existe pas dans df_facteur_risque. Assurez-vous que le script de improve_data.ipynb a été exécuté correctement.")

for i in echantillons:
    person_id = df_person.iloc[i]['person_id']

    visit = df_visit[df_visit['person_id'] == person_id].iloc[0]
    note = df_note[df_note['visit_occurrence_id'] == visit['visit_occurrence_id']].iloc[0]

    print(note['note_text'])

    res = input("Le patient est-il un fumeur? (y/n): ")
    total += 1

    fumeur = df_facteur_risque.loc[df_facteur_risque['person_id'] == person_id]['fumeur'].values

    if res == 'y':
        if fumeur: true_positif += 1
        else: faux_negatif += 1
    elif res == 'n':
        if fumeur: faux_positif += 1
        else: true_negatif += 1
    else:
        total -= 1

    clear_output(wait=True)

    print(f"Total: {total}, Faux Positifs: {faux_positif}, Faux Négatifs: {faux_negatif}, True Positifs: {true_positif}, True Négatifs: {true_negatif}")
    
    if res == 'exit':
        break

with open(os.path.join(input_dir, 'fumeur_results.txt'), 'w') as f:
    f.write(f"Total: {total}\n")
    f.write(f"Faux Positifs: {faux_positif}\n")
    f.write(f"Faux Négatifs: {faux_negatif}\n")
    f.write(f"True Positifs: {true_positif}\n")
    f.write(f"True Négatifs: {true_negatif}\n")

ValueError: La colonne 'fumeur' n'existe pas dans df_facteur_risque. Assurez-vous que le script de improve_data.ipynb a été exécuté correctement.

**Évaluation du NLP pour les consommateurs d'alcool**

In [ ]:
total = 0

faux_positif_alcool, faux_negatif_alcool = 0, 0
non_buveurs, buveurs = 0, 0

echantillons = list(range(len(df_person)))
random.shuffle(echantillons)

if 'alcool' not in df_facteur_risque.columns:
    raise ValueError("La colonne 'alcool' n'existe pas dans df_facteur_risque. Assurez-vous que le script de improve_data.ipynb a été exécuté correctement.")

for i in echantillons:
    person_id = df_person.iloc[i]['person_id']
    visit = df_visit[df_visit['person_id'] == person_id].iloc[0]
    note = df_note[df_note['visit_occurrence_id'] == visit['visit_occurrence_id']].iloc[0]

    alcoolique = df_facteur_risque.loc[df_facteur_risque['person_id'] == person_id]['alcool'].values

    if not alcoolique:
        continue # On ne considère que les patients avec un facteur de risque alcool, car il y en a trop peu

    print(note['note_text'])
    res = input("Le patient consomme-t-il de l'alcool ? (y/n) : ")

    total += 1

    if res == 'y':
        buveurs += 1
    elif res == 'n':
        faux_positif_alcool += 1
        buveurs += 1
    else:
        total -= 1

    clear_output(wait=True)

    print(f"Total : {total}, Faux positifs (alcool) : {faux_positif_alcool}, "
          f"Faux négatifs (alcool) : {faux_negatif_alcool}, Buveurs : {buveurs}, Non-buveurs : {non_buveurs}")

    if res == 'exit':
        break

if total > 10:
    with open(os.path.join(input_dir, 'alcool_results.txt'), 'w') as f:
        f.write(f"Total : {total}\n")
        f.write(f"Faux positifs (alcool) : {faux_positif_alcool}\n")
        f.write(f"Faux négatifs (alcool) : {faux_negatif_alcool}\n")
        f.write(f"Buveurs : {buveurs}\n")
        f.write(f"Non-buveurs : {non_buveurs}\n")
else:
    print("Pas assez de données pour enregistrer les résultats.")


Total : 26, Faux positifs (alcool) : 26, Faux négatifs (alcool) : 0, Buveurs : 26, Non-buveurs : 0
